## Importing Libraries

In [ ]:
import os

os.chdir("../")
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import torch
from gpytorch.kernels import (
    RBFKernel,
    ScaleKernel,
    PeriodicKernel,
    MaternKernel,
    CosineKernel,
    LinearKernel,
)
from skgpytorch.models import SVGPRegressor, SGPRegressor
from scipy.stats import kurtosis, skew
import jax.numpy as jnp
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_probability.substrates.jax as tfp

dist = tfp.distributions
import pandas as pd
import jax.numpy as jnp
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import numpy as np
from utilities import plot, errors

## Latexify

In [ ]:
try:
    from probml_utils import latexify, savefig, is_latexify_enabled
except ModuleNotFoundError:
    %pip install git+https://github.com/probml/probml-utils.git
    from probml_utils import latexify, savefig, is_latexify_enabled
os.environ["LATEXIFY"] = "1"
os.environ["FIG_DIR"] = "./Figures/"

## Data loader

In [ ]:
import pandas as pd
import jax.numpy as jnp
from datetime import datetime
from sklearn.preprocessing import StandardScaler


def dataset_load(appliances, train, test=None, bias=False):
    x_train = []
    y_train = []
    x_train_timestamp = []
    x_train_mean = []
    x_train_std = []
    x_train_max_min = []
    x_train_main = []
    x_train_max = []
    x_train_min = []
    x_train_kurtosis = []
    x_train_skew = []
    n = 99
    m = 9
    units_to_pad = n // 2
    units_to_pad1 = m // 2
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
    scaler_time = StandardScaler()
    scaler_mean = StandardScaler()
    scaler_std = StandardScaler()
    scaler_max_min = StandardScaler()
    scaler_main = StandardScaler()
    scaler_max = StandardScaler()
    scaler_min = StandardScaler()
    scaler_kurtosis = StandardScaler()
    scaler_skew = StandardScaler()

    ## train
    for key, values in train.items():
        for app in range(len(appliances)):
            df = pd.read_csv(
                f"Data/Building{key}_NILM_data_basic.csv",
                usecols=["Timestamp", "main", appliances[app]],
            )
            df["date"] = pd.to_datetime(df["Timestamp"]).dt.date
            startDate = datetime.strptime(values["start_time"], "%Y-%m-%d").date()
            endDate = datetime.strptime(values["end_time"], "%Y-%m-%d").date()

            if startDate > endDate:
                raise "Start Date must be smaller than Enddate."

            df = df[(df["date"] >= startDate) & (df["date"] <= endDate)]
            df.dropna(inplace=True)
            if app == 0:
                x = df[appliances[app]].values
            else:
                x += df[appliances[app]].values
            if appliances[app] == "Refrigerator":
                y = df[appliances[app]].values
        x_train_main.extend(x)
        timestamp_train = (
            pd.to_datetime(df["Timestamp"]).astype(int) / 10**18
        ).values
        x2 = x
        x = jnp.pad(x, (units_to_pad, units_to_pad), "constant", constant_values=(0, 0))
        x = jnp.array([x[i : i + n] for i in range(len(x) - n + 1)])
        x_train_mean.extend(jnp.mean(x, axis=1))
        x_train_std.extend(jnp.std(x, axis=1))
        x1 = jnp.pad(
            x2, (units_to_pad1, units_to_pad1), "constant", constant_values=(0, 0)
        )
        x1 = jnp.array([x1[i : i + m] for i in range(len(x1) - m + 1)])
        x_train_max_min.extend(jnp.max(x, axis=1) - jnp.min(x, axis=1))
        x_train_max.extend(jnp.max(x1, axis=1))
        x_train_min.extend(jnp.min(x1, axis=1))
        x_train_kurtosis.extend(kurtosis(x, axis=1))
        x_train_skew.extend(skew(x, axis=1))
        x_train.extend(x)
        y_train.extend(y)
        x_train_timestamp.extend(torch.tensor(timestamp_train))

    x_train = jnp.array(x_train)
    y_train = jnp.array(y_train).reshape(-1, 1)
    x_train_timestamp = torch.tensor(x_train_timestamp).reshape(-1, 1)
    x_train_main = jnp.array(x_train_main).reshape(-1, 1)
    x_train_mean = jnp.array(x_train_mean).reshape(-1, 1)
    x_train_std = jnp.array(x_train_std).reshape(-1, 1)
    x_train_max_min = jnp.array(x_train_max_min).reshape(-1, 1)
    x_train_max = jnp.array(x_train_max).reshape(-1, 1)
    x_train_skew = jnp.array(x_train_skew).reshape(-1, 1)
    x_train_min = jnp.array(x_train_min).reshape(-1, 1)
    x_train_kurtosis = jnp.array(x_train_kurtosis).reshape(-1, 1)

    x_train = scaler_x.fit_transform(x_train)
    y_train = scaler_y.fit_transform(y_train)
    x_train_timestamp = scaler_time.fit_transform(x_train_timestamp)
    x_train_main = scaler_main.fit_transform(x_train_main)
    x_train_mean = scaler_mean.fit_transform(x_train_mean)
    x_train_std = scaler_std.fit_transform(x_train_std)
    x_train_max_min = scaler_max_min.fit_transform(x_train_max_min)
    x_train_max = scaler_max.fit_transform(x_train_max)
    x_train_min = scaler_min.fit_transform(x_train_min)
    x_train_kurtosis = scaler_kurtosis.fit_transform(x_train_kurtosis)
    x_train_skew = scaler_skew.fit_transform(x_train_skew)

    # test
    x_test = []
    y_test = []
    x_test_timestamp = []
    x_test_mean = []
    x_test_std = []
    x_test_max_min = []
    x_test_timestamp_true = []
    x_test_main = []
    x_test_max = []
    x_test_min = []
    x_test_kurtosis = []
    x_test_skew = []

    for key, values in test.items():
        for app in range(len(appliances)):
            df = pd.read_csv(
                f"Data/Building{key}_NILM_data_basic.csv",
                usecols=["Timestamp", "main", appliances[app]],
            )
            df["date"] = pd.to_datetime(df["Timestamp"]).dt.date
            startDate = datetime.strptime(values["start_time"], "%Y-%m-%d").date()
            endDate = datetime.strptime(values["end_time"], "%Y-%m-%d").date()

            if startDate > endDate:
                raise "Start Date must be smaller than Enddate."

            df = df[(df["date"] >= startDate) & (df["date"] <= endDate)]
            df.dropna(inplace=True)

            if app == 0:
                x = df[appliances[app]].values
            else:
                x += df[appliances[app]].values
            if appliances[app] == "Refrigerator":
                y = df[appliances[app]].values

        if bias == True:
            x = x + 100 * np.ones(x.shape[0])

        x_test_main.extend(x)
        timestamp_true = df["Timestamp"].values
        timestamp = (pd.to_datetime(df["Timestamp"]).astype(int) / 10**18).values
        x2 = x
        x = jnp.pad(x, (units_to_pad, units_to_pad), "constant", constant_values=(0, 0))
        x = jnp.array([x[i : i + n] for i in range(len(x) - n + 1)])

        x_test_mean.extend(jnp.mean(x, axis=1))
        x_test_std.extend(jnp.std(x, axis=1))
        x1 = jnp.pad(
            x2, (units_to_pad1, units_to_pad1), "constant", constant_values=(0, 0)
        )
        x1 = jnp.array([x1[i : i + m] for i in range(len(x1) - m + 1)])
        x_test_max_min.extend(jnp.max(x, axis=1) - jnp.min(x, axis=1))
        x_test_max.extend(jnp.max(x1, axis=1))
        x_test_min.extend(jnp.min(x1, axis=1))
        x_test.extend(x)
        x_test_kurtosis.extend(kurtosis(x, axis=1))
        x_test_skew.extend(skew(x, axis=1))
        y_test.extend(y)
        x_test_timestamp_true.extend(timestamp_true)
        x_test_timestamp.extend(timestamp)

    x_test = jnp.array(x_test)
    y_test = jnp.array(y_test).reshape(-1, 1)
    x_test_timestamp = torch.tensor(x_test_timestamp).reshape(-1, 1)
    x_test_main = jnp.array(x_test_main).reshape(-1, 1)
    x_test_mean = jnp.array(x_test_mean).reshape(-1, 1)
    x_test_std = jnp.array(x_test_std).reshape(-1, 1)
    x_test_max_min = jnp.array(x_test_max_min).reshape(-1, 1)
    x_test_max = jnp.array(x_test_max).reshape(-1, 1)
    x_test_min = jnp.array(x_test_min).reshape(-1, 1)
    x_test_kurtosis = jnp.array(x_test_kurtosis).reshape(-1, 1)
    x_test_skew = jnp.array(x_test_skew).reshape(-1, 1)

    x_test = scaler_x.transform(x_test)
    x_test_timestamp = scaler_time.transform(x_test_timestamp)
    x_test_mean = scaler_mean.transform(x_test_mean)
    x_test_std = scaler_std.transform(x_test_std)
    x_test_max_min = scaler_max_min.transform(x_test_max_min)
    x_test_main = scaler_main.transform(x_test_main)
    x_test_max = scaler_max.transform(x_test_max)
    x_test_min = scaler_min.transform(x_test_min)
    x_test_kurtosis = scaler_kurtosis.transform(x_test_kurtosis)
    x_test_skew = scaler_skew.transform(x_test_skew)

    x_train = jnp.array(x_train[:, 1:]).reshape(x_train.shape[0], n - 1)
    y_train = jnp.array(y_train)
    x_train_timestamp = torch.tensor(x_train_timestamp).reshape(
        x_train_timestamp.shape[0], 1
    )
    x_test = jnp.array(x_test[:, 1:]).reshape(x_test.shape[0], n - 1)
    y_test = jnp.array(y_test)
    x_test_timestamp = (
        torch.tensor(x_test_timestamp)
        .reshape(x_test_timestamp.shape[0], 1)
        .to(torch.float64)
    )

    num_features_selected = 6
    x_train_features = jnp.concatenate(
        (
            x_train_main,
            x_train_mean,
            x_train_max_min,
            x_train_max,
            x_train_min,
            x_train_kurtosis,
        ),
        axis=1,
    ).reshape(x_train.shape[0], num_features_selected)
    x_test_features = jnp.concatenate(
        (
            x_test_main,
            x_test_mean,
            x_test_max_min,
            x_test_max,
            x_test_min,
            x_test_kurtosis,
        ),
        axis=1,
    ).reshape(x_test.shape[0], num_features_selected)

    scalers = np.array(
        [
            scaler_x,
            scaler_y,
            scaler_time,
            scaler_main,
            scaler_mean,
            scaler_std,
            scaler_max_min,
            scaler_max,
            scaler_min,
            scaler_kurtosis,
            scaler_skew,
        ]
    )
    return (
        x_train,
        y_train,
        x_test,
        y_test,
        x_train_features,
        x_test_features,
        x_train_timestamp,
        x_test_timestamp,
        scalers,
        x_test_main,
    )

In [ ]:
train = {
    1: {"start_time": "2011-04-28", "end_time": "2011-05-15"},
    3: {"start_time": "2011-04-19", "end_time": "2011-05-22"},
}

test = {
    2: {"start_time": "2011-04-21", "end_time": "2011-05-21"},
}
appliances = ["Microwave", "Refrigerator", "Dish Washer"]  #

In [ ]:
bias = False
(
    x_train,
    y_train,
    x_test,
    y_test,
    x_train_features,
    x_test_features,
    x_train_timstamp,
    x_test_timestamp,
    scalers,
    x_test_main,
) = dataset_load(appliances, train, test, bias=bias)

In [ ]:
x_train_diff = x_train_features[:, 0]
diff1 = np.array(x_train_diff)
for i in range(1, len(x_train_diff)):
		value = x_train_diff[i] - x_train_diff[i - 1]
		diff1[i] = value

x_test_diff = x_test_features[:, 0]
diff2 = np.array(x_test_diff)
for i in range(1, len(x_test_diff)):
		value = x_test_diff[i] - x_test_diff[i - 1]
		diff2[i] = value

In [ ]:
x_train_features = jnp.concatenate(
    (x_train_features, jnp.array(np.array(x_train_diff.reshape(-1, 1)))), axis=1
)
x_test_features = jnp.concatenate(
    (x_test_features, jnp.array(np.array(x_test_diff.reshape(-1, 1)))), axis=1
)

In [ ]:
n = x_train_features.shape[1]
x = torch.tensor(np.array(x_train_features)).to(torch.float64)
y = (
    torch.tensor(np.array(y_train))
    .reshape(
        -1,
    )
    .to(torch.float64)
)
xt = torch.tensor(np.array(x_test_features)).to(torch.float64)
yt = (
    torch.tensor(np.array(y_test))
    .reshape(
        -1,
    )
    .to(torch.float64)
)
x.shape, y.shape, xt.shape, yt.shape

## GP Model

In [ ]:
def GP_model(train, test, linear, ard, model_name):
    kernel1 = ScaleKernel(MaternKernel(nu=2.5, ard_num_dims=ard))
    kernel = kernel1
    if linear:
        kernel2 = ScaleKernel(LinearKernel(active_dims=(3)))
        kernel = kernel1 + kernel2
    inducing_points = x[np.arange(0, x.shape[0], 95)]

    model = SGPRegressor(x.to("cuda"), y.to("cuda"), kernel, inducing_points).to("cuda")
    if train:
        loss = model.fit(lr=1e-2, n_epochs=1500, verbose=1, random_state=0)

        plt.plot(np.asarray(loss[0]))

        model_name = model_name
        torch.save(model.state_dict(), os.path.join("./models", model_name))
    if test:
        model_name = model_name
        model.load_state_dict(torch.load(os.path.join("./models", model_name)))

    return model

In [ ]:
linear = True
if linear:
    model_name = "feature_final_linear.pt"
else:
    model_name = "feature_final.pt"

model = GP_model(train=False, test=True, linear=linear, ard=n, model_name=model_name)

## Figure 4(a)

In [ ]:
if linear:
    if bias == False:
        fig, ax = plt.subplots(1, 1)
        latexify(width_scale_factor=3, fig_height=1.75)
        ax.bar(
            range(7),
            model.mll.model.base_covar_module.kernels[0]
            .base_kernel.lengthscale.cpu()
            .detach()
            .reshape(-1, 1)[:7],
        )
        x_ticks_labels = [
            "Main",
            "Mean",
            "Range",
            "Max",
            "Min",
            "Kurtosis",
            "Difference",
        ]
        sns.despine()
        # ax.set_xlabel("Number of features")
        ax.set_ylabel("LengthScale of features")
        # ax.set_xticklabels(x_ticks_labels, rotation='vertical')
        plt.xticks(range(len(x_ticks_labels)), x_ticks_labels, rotation="vertical")
        savefig("ARD_features")

## Prediction

In [ ]:
pred_dist = model.predict((xt).to("cuda"))
y_mean = pred_dist.loc
y_mean = scalers[1].inverse_transform(y_mean.reshape(-1, 1).cpu()).squeeze()

print(y_test.shape, y_mean.shape)
y_mean = np.clip(y_mean, 0, y_mean.max(), out=y_mean)
var_pred = pred_dist.variance
var_pred = (
    scalers[1].inverse_transform(var_pred.reshape(-1, 1).detach().cpu()).squeeze()
)
std_pred = pred_dist.stddev
std_pred = torch.tensor(
    scalers[1].inverse_transform(std_pred.reshape(-1, 1).detach().cpu()).squeeze()
)

In [ ]:
mae = errors.mae(torch.tensor(y_mean), yt)
msll = errors.msll(var_pred, y_mean, yt)
qce = errors.qce(std_pred, y_mean, yt)
print("mae, msll, qce - ", mae, msll, qce)

## Figure 4 (b) and Figures 5 (c), (d)

In [ ]:
start = [4800]
idx = [200]

if bias:
    start = [4170]
    idx = [300]

x1 = x_test_features[:, 0].reshape(-1, 1)
x = scalers[3].inverse_transform(x1)
i = 0
for i in range(len(start)):
    if bias:
        if linear == False:
            plot.prediction_plots(
                x, yt, y_mean, start[i], idx[i], var_pred, "features_bias", 0
            )
        else:
            plot.prediction_plots(
                x, yt, y_mean, start[i], idx[i], var_pred, "features_bias_linear", 1
            )
    else:
        plot.prediction_plots(
            x, yt, y_mean, start[i], idx[i], var_pred, "features_1", 1
        )

## Calibration

In [ ]:
fig, ax = plt.subplots(1)
latexify(width_scale_factor=2.5, fig_height=1.75)
sigma_pred = jnp.sqrt(var_pred)
df, df1 = plot.calibration_regression(
    y_mean.squeeze(), sigma_pred.squeeze(), y_test.squeeze(), "test", "r", ax
)
ax.legend()
if bias:
    if linear:
        savefig("Features_linear_bias_calibration")
    else:
        savefig("Features_bias_calibration")
elif linear:
    savefig("Features_linear_calibration")
else:
    savefig("Features_calibration")

In [ ]:
cal = errors.find_p_hat(np.array(yt), y_mean, sigma_pred)
p = cal.index
mae_cal = errors.ace(p.values, cal.values)
print("calibration error: ", mae_cal)

## Figure 4 (c)

In [ ]:
x_lin_max = 3000
x_lin = np.linspace(0, x_lin_max, 15656)
x_time = np.linspace(
    scalers[2].inverse_transform(x_test_timestamp.reshape(-1, 1)).min(),
    scalers[2].inverse_transform(x_test_timestamp.reshape(-1, 1)).max(),
    15656,
)
x_range = np.linspace(
    scalers[6].inverse_transform(x_test_features[2].reshape(-1, 1)).min(),
    scalers[6].inverse_transform(x_test_features[2].reshape(-1, 1)).max(),
    15656,
)
x_max = np.linspace(
    scalers[7].inverse_transform(x_test_features[3].reshape(-1, 1)).min(),
    scalers[7].inverse_transform(x_test_features[3].reshape(-1, 1)).max(),
    15656,
)
x_min = np.linspace(
    scalers[8].inverse_transform(x_test_features[4].reshape(-1, 1)).min(),
    scalers[8].inverse_transform(x_test_features[4].reshape(-1, 1)).max(),
    15656,
)
x_diff = np.linspace(
    scalers[3].inverse_transform(x_test_features[5].reshape(-1, 1)).min(),
    scalers[3].inverse_transform(x_test_features[5].reshape(-1, 1)).max(),
    15656,
)
x_mean = np.linspace(
    scalers[4].inverse_transform(x_test_features[1].reshape(-1, 1)).min(),
    scalers[4].inverse_transform(x_test_features[1].reshape(-1, 1)).max(),
    15656,
)
x_kur = np.linspace(
    scalers[9].inverse_transform(x_test_features[6].reshape(-1, 1)).min(),
    scalers[9].inverse_transform(x_test_features[6].reshape(-1, 1)).max(),
    15656,
)

In [ ]:
x_lin_scale = torch.tensor(scalers[3].transform(x_lin.reshape(-1, 1))).to(torch.float32)
x_lin_dif = x_lin_scale
diff_scale = np.array(x_lin_dif)
for i in range(1, len(x_lin_dif)):
    value = x_lin_dif[i] - x_lin_dif[i - 1]
    diff_scale[i] = value

In [ ]:
x_time_scale = torch.tensor(scalers[2].transform(x_time.reshape(-1, 1))).to(
    torch.float32
)
x_range_scale = torch.tensor(diff_scale.reshape(-1, 1)).to(torch.float32)
x_max_scale = torch.tensor(scalers[7].transform(x_max.reshape(-1, 1))).to(torch.float32)
x_min_scale = torch.tensor(scalers[8].transform(x_min.reshape(-1, 1))).to(torch.float32)
x_dif_scale = torch.tensor(scalers[3].transform(x_diff.reshape(-1, 1))).to(
    torch.float32
)
x_mean_scale = torch.tensor(scalers[4].transform(x_mean.reshape(-1, 1))).to(
    torch.float32
)
x_kur_scale = torch.tensor(scalers[9].transform(x_kur.reshape(-1, 1))).to(torch.float32)

In [ ]:
x_new = torch.cat(
    (
        x_lin_scale,
        x_mean_scale,
        x_range_scale,
        x_max_scale,
        x_min_scale,
        x_kur_scale,
        x_dif_scale,
    ),
    dim=1,
).to(torch.float32)

In [ ]:
x_new.shape, x_new.dtype

In [ ]:
pred_dist = model.predict(x_new.to("cuda"))
y_mean = pred_dist.loc
y_mean = scalers[1].inverse_transform(y_mean.cpu().reshape(-1, 1))

In [ ]:
plt.figure()
latexify(width_scale_factor=2, fig_height=1.5)
start = 500
idx = 4000
plt.plot(x_lin, y_mean, "k", label=" Predicted Mean", alpha=0.7)
plt.scatter(
    scalers[3].inverse_transform(x_train_features[:, 0].reshape(-1, 1)),
    scalers[1].inverse_transform(y_train.reshape(-1, 1)),
    s=6,
    label="Appliance Power",
)
plt.xlim(00, 1500)
sns.despine()
# plt.title("Train Mains Vs Train Applaince along with Predicted Means")
plt.legend(frameon=False, bbox_to_anchor=(0.4, 0.6))
plt.xlabel("Train Mains")
plt.ylabel("Train Appliance Power")
# plt.show()
savefig("features_vs_app_mean")